In [1]:
#dependencies
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Dense, BatchNormalization, Flatten
from keras.callbacks import ModelCheckpoint
import tarfile
import dlib
import time
import os
import re
import math
from PIL import Image

Using TensorFlow backend.


In [2]:
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [3]:
def optimise_pts(x,y):
    nose_tip=[np.mean(x[30:36]),np.mean(y[30:36])]
    leye=np.asarray((np.mean(x[36:42]),np.mean(y[36:42])))
    reye=np.asarray((np.mean(x[42:48]),np.mean(y[42:48])))
    scale_dist=np.mean([np.linalg.norm(nose_tip-leye),np.linalg.norm(nose_tip-reye)])
    print(scale_dist)
    for lip_x,lip_y in zip(it.combinations(x[48:68],2),it.combinations(y[48:68],2)):#190
        x0=lip_x[0]-lip_x[1]
        y0=lip_y[0]-lip_y[1]
        dist=np.linalg.norm([x0,y0])/scale_dist*100
        print(dist)
    for leyebr_x,leyebr_y in zip(x[17:22],y[17:22]):#30
        for leye_x, leye_y in zip(x[36:42],y[36:42]):
            x1= leyebr_x - leye_x
            y1= leyebr_y - leye_y
            dist=np.linalg.norm([x1,y1])/scale_dist*100
            print(dist)
    for reyebr_x,reyebr_y in zip(x[22:27],y[22:27]):#30.....
        for reye_x, reye_y in zip(x[42:48],y[42:48]):
            x2= reyebr_x - reye_x
            y2= reyebr_y - reye_y
            dist=np.linalg.norm([x2,y2])/scale_dist*100
            print(dist)
    for jaw_x, jaw_y in zip(x[5:12],y[5:12]):#7
        x3 = jaw_x-nose_tip[0]
        y3 = jaw_y-nose_tip[1]
        dist = np.linalg.norm([x3,y3])/scale_dist*100
        print(dist)
    for nl_x,nl_y in zip(x[48:60],y[48:60]):#12==269
        x4 = nl_x-nose_tip[0]
        y4 = nl_y-nose_tip[1]
        dist = np.linalg.norm([x4,y4])/scale_dist*100
        print(dist)

In [4]:
def get_landmarks(filename):
    frame = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    detector = dlib.get_frontal_face_detector()
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(frame)
    detections = detector(clahe_image,1)
    x=[]
    y=[]
    result_data=[]
    for k,d in enumerate(detections): 
        shape = predictor(clahe_image, d)
        sub_set = []
        for i in range(1,68):
            x.append(int(shape.part(i).x))
            y.append(int(shape.part(i).y))
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2) 
            #eleminating point affecting side view
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2)
        for i in [0,1,15,16]:
            x.pop(i)
            y.pop(i)
        sub_set.append(optimise_pts(x,y))
    result_data.append(sub_set)
    cv2.imshow(filename, frame)
    #print(result_data)
    return result_data

In [5]:
import re
def classify_jaffe(filename):
    if(re.search(r'(..\.AN)', filename)):
        print(0)
        return 0
    if(re.search(r'(..\.DI)', filename)):
        print(1)
        return 1
    elif(re.search(r'(..\.FE)', filename)):
        print(2)
        return 2
    elif(re.search(r'(..\.HA)', filename)):
        print(3)
        return 3
    elif(re.search(r'(..\.SA)', filename)):
        print(4)
        return 4
    elif(re.search(r'(..\.SU)', filename)):
        print(5)
        return 5
    elif(re.search(r'(..\.NE)', filename)):
        print(6)
        return 6

In [6]:
o_Dir = "data_landmarks"
dataset_jaffe = "jaffe"
data_file = []
label=[]
#reading and getting processed data
for (dirpath, dirnames, filenames) in os.walk(dataset_jaffe):
        print(dirpath)
        for filename in filenames:
            if(re.search(r'\.tiff', filename) or re.search(r'\.jpg', filename) or re.search(r'\.png', filename) or re.search(r'\.jpeg', filename)):
                filename = dirpath+"/"+filename
                label.append(classify_jaffe(filename))
                #filenameimg = cv2.imread(dirpath+"/"+filename, cv2.IMREAD_UNCHANGED)
                #cv2.imshow(filename,filenameimg)
                data_file = get_landmarks(filename)
# writting data to o_Dir in csv format
if not os.path.exists(o_Dir):
    print("Making data_landmark directory")
    os.mkdir(o_Dir)
else:
    print("Output directory already exists")
import csv
with open(o_Dir+'/'+ dataset_jaffe+" trn_pixels", 'w') as csvfile:
    dw = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in data_file:
        for j in i:
            dw.writerow(j)
with open(o_Dir+'/'+ dataset_jaffe+" trn_label", 'w') as csvfile:
    lblw = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in label:
        lblw.writerow(j)

jaffe
0
0
0
1
1
1
2
2
2
2
3
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
1
1
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
2
2
2
3
3
3
3
6
6
6
4
4
4
4
5
5
5
0
0
0
1
1
1
2
2
2
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
1
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
1
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
1
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
1
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
0
0
0
1
1
1
2
2
2
2
3
3
3
6
6
6
4
4
4
5
5
5
Output directory already exists


In [2]:
# importing fer2016 data and landmarking
data_comp = tarfile.open("fer2013.tar") 
ds = pd.read_csv(data_comp.extractfile("fer2013/fer2013.csv"))
train = ds[["emotion", "pixels"]][ds["Usage"] == "Training"]
train['pixels'] = train['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
train_pix = np.vstack(train['pixels'].values)
test = ds[["emotion", "pixels"]][ds["Usage"] == "PublicTest"]
test['pixels'] = test['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
test_pix = np.vstack(test['pixels'].values)
train_ind = np.array(train["emotion"])
test_ind = np.array(test["emotion"])

[  70.   80.   82. ...,  106.  109.   82.]


In [4]:
o_Dir = "data_landmarks"
dataset_jaffe = "jaffe"
data_file = []
label=[]
train_pix = train_pix.reshape(-1,48,48,1)
filename = Image.fromarray(x).convert('RGB')
train_shape = train_pix.shape
for i in range(train_shape[0]):
    data_file = get_landmarks(filename)
import csv
with open(o_Dir+'/'+ dataset_jaffe+" trn_pixels", 'w') as csvfile:
    dw = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in data_file:
        for j in i:
            dw.writerow(j)
with open(o_Dir+'/'+ dataset_jaffe+" trn_label", 'w') as csvfile:
    lblw = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in label:
        lblw.writerow(j)

NameError: name 'train_pix' is not defined

((28709, 7), (3589, 7))

In [ ]:
#Set up some required objects
video_capture = cv2.VideoCapture(0) #Webcam object
#detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #Landmark identifier. Set the filename to whatever you named the downloaded file
###
ap = argparse.ArgumentParser()
ap.add_argument("-c", "--cascade", required=True, help = "haarcascade_frontalface_default.xml")
#ap.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")
args = vars(ap.parse_args())

detector = cv2.CascadeClassifier(args["cascade"])
#predictor = dlib.shape_predictor(args["shape_predictor"])

#facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

fno=0
detections=()
while True:
    ret, frame = video_capture.read()
    #frame=cv2.imread('a.jpg',1)
    fno+=1;
    if(False):
        pass
    else:
        print(fno)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        clahe_image = clahe.apply(gray)
        #faces = facec.detectMultiScale(clache_image, 1.3, 5)
        # for (x, y, w, h) in faces:
         #   fc = gray_fr[y:y+h, x:x+w]
          #  roi = cv2.resize(fc, (48, 48))
        detections = detector(clahe_image, 1) #Detect the faces in the image
        #print(detections)
    for k,d in enumerate(detections): #For each detected face
        shape = predictor(clahe_image, d) #Get coordinates
        for i in range(1,68): #There are 68 landmark points on each face
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2) #For each point, draw a red circle with thickness2 on the original frame
    cv2.imshow("image", frame) #Display the frame
    
    if cv2.waitKey(1) == 27: #Exit program when the user presses 'q'
        break